In [ ]:
import sqlite3
import tensorflow as tf
import numpy as np
import cv2
import json
from werkzeug.wrappers import Request, Response
from flask import Flask, request, jsonify,render_template
from flask_cors import CORS
import requests
from bs4 import BeautifulSoup
import re
import base64
import os
os.chdir('C:\\Users\\Ash347\\Downloads\\server')
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2490.80 Safari/537.36',
    'Content-Type': 'text/html',
}
conn = sqlite3.connect('database.db')
cursor=conn.cursor()
with open('C:\\Users\\Ash347\\Downloads\\server\\cat.json') as f:
  data = json.load(f)
model = tf.keras.models.load_model('C:\\Users\\Ash347\\Downloads\\server\\first.model')
app = Flask(__name__)
CORS(app)
def extractdata(flower_name):
    flower_name=flower_name.strip()
    flower_name=flower_name.title()
    flower=flower_name
    resp="|"
    cursor.execute("SELECT * from plantdata where comname='"+flower_name+"'")
    row = cursor.fetchone()
    if row is not None:
        return resp.join(row)
    flowers=os.listdir('flowers')
    flower_name=flower_name.replace(' ','%20')
    url="http://www.flowersofindia.net/catalog/slides/"+flower_name+".html"
    response = requests.get(url,headers=headers)    
    parse = BeautifulSoup(response.text, "html.parser")
    description_withtags=parse.find('div',{'id':'descr'})
    if(description_withtags==None):
        return flower_name
    else:
        description=re.sub('<[^<]+?>', '', str(description_withtags))
        family_withtags=parse.find('div',{'id':'botname'})
        family=re.sub('<[^<]+?>', '', str(family_withtags))
        img_data = requests.get("http://www.flowersofindia.net/catalog/slides/"+flower_name+".jpg").content
        with open('flowers/'+flower_name+".jpg", 'wb') as handler:
            handler.write(img_data)
        imageFile= open("flowers/"+flower_name+".jpg", "rb")
        imagestring = base64.b64encode(imageFile.read())
        resp="|"
        values= family.split("\xa0\xa0\xa0")+[description,imagestring.decode('utf-8'),flower]
        for i in range(len(values)):
            values[i]=values[i].replace("'","")
        cursor.execute("INSERT INTO plantdata(comname,botname,family,description,image) VALUES ('"+flower+"','"+values[0]+"','"+values[1]+"','"+values[2]+"','"+values[3]+"');")
        conn.commit()
        return resp.join(values)
@app.route('/insertsearch',methods=['POST','GET'])
def insertsearch():
    email=request.args.get('email');
    data=request.args.get('data');
    cursor.execute("SELECT * FROM searchdata WHERE name='"+email+"'")
    row=cursor.fetchall()
    data=data.strip()
    data=data.title()
    if(len(row)>=1):
        history=row[0][1]
        searches=history.split(',')
        if data not in searches:
            history=history+','+data
            cursor.execute("UPDATE searchdata SET history='"+history+"' WHERE name='"+email+"'")
    else:
        cursor.execute("INSERT INTO searchdata (name,history) \
          VALUES ('"+email+"','"+data+"')")
    conn.commit()
    return "yes"
@app.route('/retrievesearch',methods=['POST','GET'])
def retrievesearch():
    email=request.args.get('email')
    cursor.execute("SELECT * FROM searchdata WHERE name='"+email+"'")
    row=cursor.fetchall()
    if(len(row)==1):
        history=row[0][1]
        names=history.split(',')
        exportdata=""
        for name in names:
            details=extractdata(name)
            exportdata=exportdata+"^"+details
        return exportdata
    else:
        return "nil"
@app.route('/main',methods=['POST','GET'])
def index():
    return render_template('1.htm')

@app.route('/new12',methods=['POST','GET'])
def new12():
    return render_template('new12.htm')
@app.route('/recent',methods=['POST','GET'])
def recent():
    return render_template('recent.html')
@app.route('/newrose',methods=['POST','GET'])
def newrose():
    return render_template('newrose.html')

@app.route('/classify',methods=['POST','GET'])
def classify():
    if request.method=='POST':
        file=request.files['img']
        file.save(file.filename)
        img=cv2.imread(file.filename)
        img = cv2.resize(img, (96,96))
        img=img.reshape(1,96,96,3)
        img=img.astype('float32')
        predictions=model.predict(img)
        flower_class=np.argmax(predictions)
        flower_name=data[str(flower_class)]
    else:
        flower_name=request.args.get('flowername')
    flower_details=extractdata(flower_name)
    return flower_details
@app.route('/signup',methods=['GET','POST'])
def signup():
    name=request.args.get('name')
    password=request.args.get('password')
    email=request.args.get('email')
    cursor.execute("SELECT * FROM users WHERE email='"+email+"'")
    if(len(cursor.fetchall())>=1):
        return "false"
    else:
        cursor.execute("INSERT INTO users (name,email,password) \
          VALUES ('"+name+"','"+email+"','"+password+"')")
        conn.commit()
        return "yes"

@app.route('/signin',methods=['GET','POST'])
def signin():
    password=request.args.get('password')
    email=request.args.get('email')
    cursor.execute("SELECT name,email from users where email='"+email+"' and password='"+password+"'")
    row = cursor.fetchone()
    if row is None:
        return "no"
    else:
        return row[0]
# HTTP Errors handlers
@app.errorhandler(404)
def url_error(e):
    return """
    Wrong URL!
    <pre>{}</pre>""".format(e), 404

@app.errorhandler(500)
def server_error(e):
    return """
    An internal error occurred: <pre>{}</pre>
    See logs for full stacktrace.
    """.format(e), 500
if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 8050, app)

 * Running on http://localhost:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [23/Mar/2020 10:01:34] "POST /classify HTTP/1.1" 200 -
127.0.0.1 - - [23/Mar/2020 10:17:37] "POST /classify HTTP/1.1" 200 -
127.0.0.1 - - [23/Mar/2020 16:24:27] "GET /signin?email=akr&password=akr HTTP/1.1" 200 -


In [25]:
email='akr'
data='rusty acacia'
cursor.execute("SELECT * FROM searchdata WHERE name='"+email+"'")
row=cursor.fetchall()
print(row)
data=data.strip()
data=data.title()
if(len(row)>=1):
    history=row[0][1]
    searches=history.split(',')
    if data not in searches:
        history=history+','+data
        cursor.execute("UPDATE searchdata SET history='"+history+"' WHERE name='"+email+"'")
else:
    cursor.execute("INSERT INTO searchdata (name,history) \
      VALUES ('"+email+"','"+data+"')")
conn.commit()

[('akr', 'Indian Mallow, Persian Mallow, Painted Mallow,Amazon Blue,Rusty Acacia')]
c


In [5]:
import sqlite3
import numpy as np
import cv2
import json
from werkzeug.wrappers import Request, Response
from flask import Flask, request, jsonify,render_template
from flask_cors import CORS
import requests
from bs4 import BeautifulSoup
import re
import base64
import os
os.chdir('C:\\Users\\Ash347\\Downloads\\server')
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2490.80 Safari/537.36',
    'Content-Type': 'text/html',
}
conn = sqlite3.connect('database.db')
cursor=conn.cursor()
def extractdata(flower_name):
    flower_name=flower_name.strip()
    flower_name=flower_name.title()
    flower=flower_name
    resp="|"
    cursor.execute("SELECT * from plantdata where comname='"+flower_name+"'")
    row = cursor.fetchone()
    if row is not None:
        return resp.join(row)
    flowers=os.listdir('flowers')
    flower_name=flower_name.replace(' ','%20')
    url="http://www.flowersofindia.net/catalog/slides/"+flower_name+".html"
    response = requests.get(url,headers=headers)    
    parse = BeautifulSoup(response.text, "html.parser")
    description_withtags=parse.find('div',{'id':'descr'})
    if(description_withtags==None):
        return flower_name
    else:
        description=re.sub('<[^<]+?>', '', str(description_withtags))
        family_withtags=parse.find('div',{'id':'botname'})
        family=re.sub('<[^<]+?>', '', str(family_withtags))
        img_data = requests.get("http://www.flowersofindia.net/catalog/slides/Ladies%20Finger.jpg").content
        with open('flowers/'+flower_name+".jpg", 'wb') as handler:
            handler.write(img_data)
        imageFile= open("flowers/"+flower_name+".jpg", "rb")
        imagestring = base64.b64encode(imageFile.read())
        print(imagestring)
        resp="|"
        values= family.split("\xa0\xa0\xa0")+[description,imagestring.decode('utf-8'),flower]
        for i in range(len(values)):
            values[i]=values[i].replace("'","")
        cursor.execute("INSERT INTO plantdata(comname,botname,family,description,image) VALUES ('"+flower+"','"+values[0]+"','"+values[1]+"','"+values[2]+"','"+values[3]+"');")
        conn.commit()
        return resp.join(values)
data = extractdata('Indian Mallow')

no
